# Carga Datos JJA

In [1]:
# Cargamos paquetes

library(repr)
library(dplyr)

library(abind)
library(loadeR)
library(transformeR)
library(visualizeR)
library(downscaleR)
library(climate4R.UDG)
library(climate4R.climdex)
library(climate4R.indices)
library(easyVerification)

library(lattice)
library(magrittr)
library(gridExtra)
library(RColorBrewer)

# biasCorrection_RM
source("load_bc_functions.R")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: rJava

Loading required package: loadeR.java

Java version 23x amd64 by N/A detected

NetCDF Java Library v4.6.0-SNAPSHOT (23 Apr 2015) loaded and ready

Loading required package: climate4R.UDG

climate4R.UDG version 0.2.6 (2023-06-26) is loaded

Please use 'citation("climate4R.UDG")' to cite this package.

loadeR version 1.8.1 (2023-06-22) is loaded


Get the latest stable version (1.8.2) using <devtools::install_github(c('SantanderMetGroup/climate4R.UDG','SantanderMetGroup/loadeR'))>

Please use 'citation("loadeR")' to cite this package.




    _______   ____  ___________________  __  ________ 
   / ___/ /  / /  |/  / __  /_  __/ __/ / / / / __  / 
  / /  / /  / / /|_/ / /_/ / / / / __/ / /_/ / /_/_/  
 / /__/ /__/ / /  / / __  / / / / /__ /___  / / \ \ 
 \___/____/_/_/  /_/_/ /_/ /_/  \___/    /_/\/   \_\ 
 
      github.com/SantanderMetGroup/climate4R



transformeR version 2.2.2 (2023-10-26) is loaded


Get the latest stable version (2.2.3) using <devtools::install_github('SantanderMetGroup/transformeR')>

Please see 'citation("transformeR")' to cite this package.

visualizeR version 1.6.4 (2023-10-26) is loaded

Please see 'citation("visualizeR")' to cite this package.

downscaleR version 3.3.4 (2023-06-22) is loaded

Please use 'citation("downscaleR")' to cite this package.

Loading required package: climdex.pcic

Loading required package: PCICt

climate4R.climdex version 0.2.3 (2023-06-23) is loaded

Use 'climdexShow()' for an overview of the available ETCCDI indices

climate4R.indices version 0.3.1 (2023-06-22) is loaded

Use 'indexShow()' for an overview of the available climate indices and circIndexShow() for the circulation indices.

NOTE: use package climate4R.climdex to calculate ETCCDI indices.


Attaching package: ‘climate4R.indices’


The following object is masked from ‘package:transformeR’:

    lambWT


Loading required

In [2]:
# Region de estudio

lon = c(-10, 20)
lat = c(35,46)

## Cargar hindcast (leadtime 0, 1, 2, 3)

In [3]:
anios = 1981:2016
meses_ini = c("06", "05", "04", "03")  # meses de inicialización

# Función generalizada para cargar los datos por mes y año
cargar_dato = function(anio, mes_ini) {
    
    yyyymm = paste0(anio, mes_ini)

    ruta = paste0(
        "/lustre/gmeteo/PTICLIMA/DATA/SEASONAL/",
        "seasonal-original-single-levels/medcof/hindcast/tasmax/ecmwf/51/", yyyymm, "/",
        "seasonal-original-single-levels_medcof_hindcast_tasmax_ecmwf_51_", yyyymm, ".ncml"
    )

    data_aux = loadGridData(dataset = ruta,
                            var = "tasmax",
                            lonLim = lon,
                            latLim = lat,
                            season = c(6, 7, 8)) %>% suppressMessages %>% suppressWarnings

    data_aux = aggregateGrid(data_aux, aggr.d = list(FUN = "mean", na.rm = TRUE)) %>% suppressMessages %>% suppressWarnings

    return(data_aux)
}

# Creo una lista donde cada elemento es la salida para un mes de inicialización diferente
resultados_por_mes = lapply(meses_ini, function(mes) {
    lapply(anios, function(anio) cargar_dato(anio, mes))
})

# Nombro los elementos por mes
names(resultados_por_mes) = paste0("mes_", meses_ini)

In [4]:
# Asigno nombre a cada grid con los leadtime
tmx_ecmwf_jja_0 = resultados_por_mes[["mes_06"]]
tmx_ecmwf_jja_1 = resultados_por_mes[["mes_05"]]
tmx_ecmwf_jja_2 = resultados_por_mes[["mes_04"]]
tmx_ecmwf_jja_3 = resultados_por_mes[["mes_03"]]

In [5]:
# Combinamos los grids en la dimensión temporal
tmx_ecmwf_jja_0_grid = bindGrid(tmx_ecmwf_jja_0, dimension = "time")
tmx_ecmwf_jja_1_grid = bindGrid(tmx_ecmwf_jja_1, dimension = "time")
tmx_ecmwf_jja_2_grid = bindGrid(tmx_ecmwf_jja_2, dimension = "time")
tmx_ecmwf_jja_3_grid = bindGrid(tmx_ecmwf_jja_3, dimension = "time")

## Cargar forecast (leadtime 0, 1, 2, 3)

In [6]:
anios = 2017:2022
meses_ini = c("06", "05", "04", "03")  # meses de inicialización

# Función generalizada para cargar los datos por mes y año
cargar_dato = function(anio, mes_ini) {
    
    yyyymm = paste0(anio, mes_ini)

    ruta = paste0(
        "/lustre/gmeteo/PTICLIMA/DATA/SEASONAL/",
        "seasonal-original-single-levels/medcof/forecast/tasmax/ecmwf/51/", yyyymm, "/",
        "seasonal-original-single-levels_medcof_forecast_tasmax_ecmwf_51_", yyyymm, ".ncml"
    )

    data_aux = loadGridData(dataset = ruta,
                            var = "tasmax",
                            lonLim = lon,
                            latLim = lat,
                            season = c(6, 7, 8)) %>% suppressMessages %>% suppressWarnings

    data_aux = aggregateGrid(data_aux, aggr.d = list(FUN = "mean", na.rm = TRUE)) %>% suppressMessages %>% suppressWarnings

    return(data_aux)
}

# Creo una lista donde cada elemento es la salida para un mes de inicialización diferente
forecast = lapply(meses_ini, function(mes) {
    lapply(anios, function(anio) cargar_dato(anio, mes))
})

# Nombro los elementos por mes
names(forecast) = paste0("mes_", meses_ini)

In [7]:
# Asigno nombre a cada grid con los leadtime
tmx_forecast_jja_0 = forecast[["mes_06"]]
tmx_forecast_jja_1 = forecast[["mes_05"]]
tmx_forecast_jja_2 = forecast[["mes_04"]]
tmx_forecast_jja_3 = forecast[["mes_03"]]

In [8]:
# Combinamos los grids en la dimensión temporal
tmx_forecast_jja_0_grid = bindGrid(tmx_forecast_jja_0, dimension = "time")
tmx_forecast_jja_1_grid = bindGrid(tmx_forecast_jja_1, dimension = "time")
tmx_forecast_jja_2_grid = bindGrid(tmx_forecast_jja_2, dimension = "time")
tmx_forecast_jja_3_grid = bindGrid(tmx_forecast_jja_3, dimension = "time")

In [9]:
# Me quedo con los primeros 25 miembros
tmx_forecast_jja_0_members = subsetGrid(tmx_forecast_jja_0_grid, members = 1:25)
tmx_forecast_jja_1_members = subsetGrid(tmx_forecast_jja_1_grid, members = 1:25)
tmx_forecast_jja_2_members = subsetGrid(tmx_forecast_jja_2_grid, members = 1:25)
tmx_forecast_jja_3_members = subsetGrid(tmx_forecast_jja_3_grid, members = 1:25)

In [10]:
# Combinamos los grids de hindcast y forecast
final_0_grid = bindGrid(tmx_ecmwf_jja_0_grid, tmx_forecast_jja_0_members, dimension = "time")
final_1_grid = bindGrid(tmx_ecmwf_jja_1_grid, tmx_forecast_jja_1_members, dimension = "time")
final_2_grid = bindGrid(tmx_ecmwf_jja_2_grid, tmx_forecast_jja_2_members, dimension = "time")
final_3_grid = bindGrid(tmx_ecmwf_jja_3_grid, tmx_forecast_jja_3_members, dimension = "time")

## Cargar las observaciones ERA5-Land

In [11]:
# Define años y meses
anios = 1981:2022
meses = sprintf("%02d", 6:8)

# Función para construir la ruta y cargar los datos
cargar_dato = function(anio, mes) {
    yyyy = paste0(anio)
    yyyymm = paste0(anio, mes)
    ruta = paste0(
        "/lustre/gmeteo/PTICLIMA/DATA/REANALYSIS/ERA5-Land/data/Iberia/day/t2mx/", yyyy, "/",
        "t2mx_ERA5-Land_", yyyymm, ".nc"
    )
    
    # Carga el dataset
    data_aux = loadGridData(dataset = ruta,
                            var = "t2mx",
                            lonLim = lon,
                            latLim = lat,
                            aggr.d = 'mean') %>% suppressMessages %>% suppressWarnings
    
    # Cargo datos diario para trabajar con biasCorrection, habría que agregar aggr.m = 'mean' para el dato mensual

    return(data_aux)
}

# Crear combinaciones año-mes
combinaciones = data.frame(
  anio = rep(anios, each = 3),
  mes = rep(meses, times = length(anios)),
  stringsAsFactors = FALSE
)

# Aplicar la función a cada combinación
tmx_era5_jja = lapply(1:nrow(combinaciones), function(i) {
  cargar_dato(combinaciones$anio[i], combinaciones$mes[i])
})

In [12]:
# Combinamos los grids en la dimensión temporal
tmx_era5_jja_time = bindGrid(tmx_era5_jja, dimension = "time")

# Upscaling de la resolución de las observaciones
tmx_era5_jja_ups = interpGrid(tmx_era5_jja_time,
                              new.coordinates = getGrid(tmx_ecmwf_jja_0_grid),
                              method = "bilinear") %>% suppressMessages %>% suppressWarnings

In [13]:
# Recorto los datos a la región de España
tmx_modelo_0 = subsetGrid(final_0_grid, lonLim = c(-10, 5), latLim = c(35, 44))
tmx_modelo_1 = subsetGrid(final_1_grid, lonLim = c(-10, 5), latLim = c(35, 44))
tmx_modelo_2 = subsetGrid(final_2_grid, lonLim = c(-10, 5), latLim = c(35, 44))
tmx_modelo_3 = subsetGrid(final_3_grid, lonLim = c(-10, 5), latLim = c(35, 44))

tmx_obs = subsetGrid(tmx_era5_jja_ups, lonLim = c(-10, 5), latLim = c(35, 44))

# Pasamos las observaciones de Kelvin a Celsius
tmx_obs_cel = gridArithmetics(tmx_obs, 273.15, operator = "-")

In [14]:
getShape(tmx_modelo_0)
getShape(tmx_obs_cel)

member   time    lat    lon 
    25   3864     10     16

time  lat  lon 
3864   10   16

In [15]:
# Guardo los datos
saveRDS(tmx_modelo_0, file = "tmx_modelo_jja_0.rds")
saveRDS(tmx_modelo_1, file = "tmx_modelo_jja_1.rds")
saveRDS(tmx_modelo_2, file = "tmx_modelo_jja_2.rds")
saveRDS(tmx_modelo_3, file = "tmx_modelo_jja_3.rds")

saveRDS(tmx_obs_cel, file = "tmx_obs_cel_jja.rds")